In [115]:
import pandas as pd
field_data = pd.read_csv('toy_fields.csv') #data about the size of the fields in acres
org_data = pd.read_csv('toy_orgs.csv') #data about the names of the orgs and their unique id's
yield_data = pd.read_csv('toy_yields.csv') #data about the harvest details 
#field_data.info()
#org_data.info()
#yield_data.info()
yield_data.head()

,crop_type,year,harvest_product,total_harvested,harvested_unit,planted_area,harvested_area,field_id,org_id
0,CROP.CORN.COMMERCIAL,2016,grain,53290.628852,bu,40.380000,560.0,cb1e3bd821dd673a0c378b16ba97075919aa993a,0966e34316e39217e4f26fbe20fc14e24c26567b
1,CROP.CORN.COMMERCIAL,2014,grain,40073.821554,bu,NaN,NaN,58d04a7094c1b08e9856932a02104d2a36b7b8ce,726cce9d9800d792d1e0fbb83923ac4c6be4f668
2,CROP.CORN.COMMERCIAL,2015,grain,45528.033941,bu,148.610000,NaN,5e3babe7b318f4245ceb16718665af0b54335443,598d17ec200eeb9e27ae829746ccf47c57761bc1
3,CROP.SOYBEAN.COMMERCIAL,2015,grain,13927.332280,bu,25.000000,NaN,02dc3fe6f889f3ece8daf456d26464d381bc9e55,fe62dee029474d52346542845674529025594056
4,CROP.CORN.COMMERCIAL,2014,grain,59676.634874,bu,5.026942,NaN,109e500b323e0ef7baadd4409830c7619d8cfbfa,fe62dee029474d52346542845674529025594056


#### hm. right away I can see some NaN values that warrant some attention. The missing entries tells me that the quality of this data quality is imperfect. I'll have to decide later on whether I want to discard those specific data rows or fill them in somehow. Also, another point of concern is the planted area and harvested area values. In the 1st row, 'planted_area' should not exceed the value of 'harvested_area'. The quality of this data may prove to be invalid should these entries are unable to be handled. 

In [138]:
updated_harvest_area = []
updated_planted_area = []
for index,row in yield_data.iterrows():
    if row['harvested_area']>row['planted_area']:
        updated_harvest_area.append(None)
        updated_planted_area.append(None)
    else:
        updated_harvest_area.append(row['harvested_area'])
        updated_planted_area.append(row['planted_area'])
         
yield_data['cleaned_planted_area'] = updated_planted_area
yield_data['cleaned_harvest_area'] = updated_harvest_area  


#### With the above forloop, I create a new column called 'updated_harvest_area' to clean the data of harvest areas that exceeded the planted area

In [139]:
yield_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 2 to 416
Data columns (total 11 columns):
crop_type               415 non-null object
year                    415 non-null int64
harvest_product         415 non-null object
total_harvested         415 non-null float64
harvested_unit          415 non-null object
field_id                415 non-null object
org_id                  415 non-null object
cleaned_planted_area    228 non-null float64
cleaned_harvest_area    131 non-null float64
planted_area            304 non-null float64
harvested_area          207 non-null float64
dtypes: float64(5), int64(1), object(5)
memory usage: 35.7+ KB


#### There were many missing values in the planted area column and the harvested area column. After the data cleaning, 77 of both the planted_area and harvest_area entries were cleared and only (228,131) respective valid entries are remaining. 

In [140]:
org_data.rename(columns = {'new_org_id': 'org_id'}, inplace= True)
org_data.columns
cols = yield_data.columns.tolist() # just rearranging column to make is easier to analyze
cols= cols[:5]+cols[7:]+ cols[5:7]
yield_data = yield_data[cols]
yield_data

,crop_type,year,harvest_product,total_harvested,harvested_unit,cleaned_planted_area,cleaned_harvest_area,planted_area,harvested_area,field_id,org_id
2,CROP.CORN.COMMERCIAL,2015,grain,45528.033941,bu,148.610000,NaN,148.610000,NaN,5e3babe7b318f4245ceb16718665af0b54335443,598d17ec200eeb9e27ae829746ccf47c57761bc1
3,CROP.SOYBEAN.COMMERCIAL,2015,grain,13927.332280,bu,25.000000,NaN,25.000000,NaN,02dc3fe6f889f3ece8daf456d26464d381bc9e55,fe62dee029474d52346542845674529025594056
4,CROP.CORN.COMMERCIAL,2014,grain,59676.634874,bu,5.026942,NaN,5.026942,NaN,109e500b323e0ef7baadd4409830c7619d8cfbfa,fe62dee029474d52346542845674529025594056
5,CROP.CORN.COMMERCIAL,2014,grain,67817.792347,bu,124.050462,NaN,124.050462,NaN,a1b95c0d5d12da17a82e1af370743f9b174574e0,106e6791d0f181b391cde800e74fdc14d529dc0a
6,CROP.SOYBEAN.COMMERCIAL,2017,grain,16899.041092,bu,NaN,NaN,NaN,NaN,96086a4a633ca9059f8c0472c7e532c14606294a,fe62dee029474d52346542845674529025594056
7,CROP.CORN.COMMERCIAL,2017,grain,74801.703591,bu,32.780000,NaN,32.780000,NaN,d1f129caa6ad88c0ca2b6b274dbb225c3aa3e5f8,fe62dee029474d52346542845674529025594056
8,CROP.SOYBEAN.COMMERCIAL,2015,grain,30341.901524,bu,80.210000,NaN,80.210000,NaN,ed7e44f184a071a25c905e1ee9f73c6741e0bd01,598d17ec200eeb9e27ae829746ccf47c57761bc1
9,CROP.CORN.COMMERCIAL,2015,grain,50987.861543,bu,179.120000,NaN,179.120000,NaN,fd72b33016dc565f97e287594573282d0ae29f08,598d17ec200eeb9e27ae829746ccf47c57761bc1
10,CROP.CORN.COMMERCIAL,2016,grain,4109.418402,bu,249.850000,65.000000,249.850000,65.000000,ceacbff5f896d6b9f5e1c7d3a520f1e832806158,b3bd5efa1dbecf1a1a564b8bc6303c4a2c162fb1
11,CROP.SOYBEAN.COMMERCIAL,2015,grain,403.185902,bu,57.820000,57.820000,57.820000,57.820000,0c090e104b85e5d305640f9d43f32d0a8c5faeb6,0966e34316e39217e4f26fbe20fc14e24c26567b


#### we have to rename the 'organization_id' column in our toy_org.csv file to 'org_id' to be able to join the two data frames

In [128]:
org_data.head()

,grower,organization_id,org_id
0,Cavanaugh Farm,bf08b038-921c-482f-85b7-17c52aa98095,fe62dee029474d52346542845674529025594056
1,Depner Farm,c0d679bb-50c3-40b2-8a6b-313b6010aae6,106e6791d0f181b391cde800e74fdc14d529dc0a
2,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3,0966e34316e39217e4f26fbe20fc14e24c26567b
3,Batte Farm,8bbd1e4f-8783-433d-ac74-27e81d67fb43,513612757cd6094d98b6a82a01d6f115fe9ae88c
4,Sifuentes Farm,35cedd2e-a117-4713-983f-3574fffce422,2ec53b20e3a9f6ff73d290ef3e8e8f30a2a30235


In [120]:
mergedDf = pd.merge(yield_data,org_data, left_on = ['org_id'],right_on = ['org_id'])
mergedDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 0 to 416
Data columns (total 13 columns):
crop_type               417 non-null object
year                    417 non-null int64
harvest_product         417 non-null object
total_harvested         417 non-null float64
harvested_unit          417 non-null object
planted_area            305 non-null float64
harvested_area          208 non-null float64
field_id                417 non-null object
org_id                  417 non-null object
cleaned_planted_area    228 non-null float64
cleaned_harvest_area    131 non-null float64
grower                  417 non-null object
organization_id         417 non-null object
dtypes: float64(5), int64(1), object(7)
memory usage: 45.6+ KB


#### After joining the yield data and org names, I see a lot of missing values to where the harvested yields came from. This further provides evidence that there was improper handling during data collection as dataset does not indicate where the harvest has come from.

In [125]:
mergedDf.grower.value_counts()

Simmons Farm        148
Cavanaugh Farm       82
Dudley Farm          51
Austin Farm          50
Depner Farm          34
Konecny Farm          8
Jenkins Farm          7
Odell Farm            6
Hawker Farm           5
Ross Farm             4
Gregory Farm          4
Overholt Farm         3
Flanagan Farm         3
Sifuentes Farm        2
Mull Farm             2
Seever Farm           2
Barrocas Farm         2
Carpentieri Farm      1
Crawford Farm         1
Batte Farm            1
Stinson Farm          1
Name: grower, dtype: int64

In [126]:
mergedDf.loc[mergedDf['grower']=='Dudley Farm'].year.value_counts()

2015    33
2014    18
Name: year, dtype: int64

#### With this I can tell that several entries are created for the same year, that could have otherwise been compacted. this can most likely be compacted using the groupby function and specificying which featured we want to see the compacted values for. 

In [127]:
mergedDf.groupby(['grower','year'])['total_harvested','planted_area','harvested_area'].sum()

total_harvested  planted_area  harvested_area
grower           year                                               
Austin Farm      2015     2.038052e+06   4152.140000     4159.310000
                 2016     2.147659e+05    118.500000      830.000000
Barrocas Farm    2017     1.174206e+05     95.000000      294.000000
Batte Farm       2017     7.047084e+04      0.000000       13.000000
Carpentieri Farm 2016     7.119898e+04      0.000000        0.000000
Cavanaugh Farm   2014     3.929600e+05    155.531958      469.200000
                 2015     1.589323e+06    380.285906     1544.010000
                 2016     1.144137e+06    730.520000      598.662117
                 2017     1.266030e+06    732.750000     1418.840000
Crawford Farm    2017     9.469835e+04      0.000000      150.000000
Depner Farm      2014     5.356606e+05   1203.275551     1199.570000
                 2015     6.691972e+05    611.979999       95.300000
                 2016     2.044040e+05      0.000000      387.800000
                 2017     3.375841e+05    105.532126      200.000000
Dudley Farm      2014     8.432750e+05      0.000000      378.000000
                 2015     1.218157e+06   3775.001500      843.500000
Flanagan Farm    2015     4.109234e+04    100.630000        0.000000
                 2016     3.839162e+04     77.320000        0.000000
Gregory Farm     2015     1.991505e+05      0.000000      309.500000
                 2017     2.069695e+04      0.000000        0.000000
Hawker Farm      2015     7.485965e+04    114.750000      114.390000
                 2016     3.674980e+04    249.850000      144.000000
                 2017     6.697064e+04      0.000000        0.000000
Jenkins Farm     2016     2.321079e+05    364.267784      216.240000
Konecny Farm     2017     3.269919e+05      0.000000      184.000000
Mull Farm        2017     4.219978e+04      0.000000      296.000000
Odell Farm       2014     1.784253e+05     48.300000      279.000000
                 2016     9.191359e+04      0.000000        0.000000
                 2017     1.046427e+05     10.000000        0.000000
Overholt Farm    2017     2.207304e+05     60.000000        0.000000
Ross Farm        2017     1.619524e+05    209.574175      209.290000
Seever Farm      2017     2.226073e+05     56.000000        0.000000
Sifuentes Farm   2017     4.454520e+04      0.470000        0.000000
Simmons Farm     2015     2.582268e+06   5473.190000      290.550000
                 2016     2.221029e+06   4848.298441     4995.802982
                 2017     1.738078e+06   4498.798784     3503.124000
Stinson Farm     2017     8.186285e+04     10.490000       10.490000

## The data quality is seemingly erroneous and the dataset cannot be trusted with full confidence. That said, it is still possible to be able to compare valid yield data between the farms and see 